In [1]:
%pip install lightning

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U 'wandb>=0.12.10'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torchvision.models
import torchvision.transforms as transforms

from datapipeline.asl_image_data_module import ASLImageDataModule
from lightning.pytorch.loggers import WandbLogger
from models.asl_model import ASLModel
from models.training import train, sweep, PROJECT_NAME, ENTITY_NAME
import wandb

In [4]:
img_size = 224

data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
datamodule = ASLImageDataModule(path="/exchange/dspro2/silent-speech/ASL_Pictures_Dataset", transforms=data_transforms)

In [5]:
class ASLCNN_img(nn.Module):
    def __init__(self, kernel_size, dropout=0.2, hidden_dim=128):
        super().__init__()
        self.kernel_size = kernel_size
        self.hidden_dim = hidden_dim
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=self.kernel_size, stride=2, padding=3), # 400 -> 200
            nn.ReLU(),
            nn.MaxPool2d(2),  # 200 -> 100
            
            nn.Conv2d(32, 64, kernel_size=self.kernel_size, padding=1),  # 100
            nn.ReLU(),
            nn.MaxPool2d(2),  # 100 -> 50
            
            nn.Conv2d(64, 128, kernel_size=self.kernel_size, padding=1), # 50
            nn.ReLU(),
            nn.MaxPool2d(2),  # 50 -> 25
        
            nn.Flatten(),
            nn.LazyLinear(self.hidden_dim), #nn.Linear(128 * 25 * 25, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.hidden_dim, 28),  # number of classes
        )

    def forward(self, x):
        x = self.model(x)
        return x



In [6]:
sweep_config = {
    "name": "cnn-kernel-lr-dropout-sweep",
    "method": "bayes",  # or "grid", "random"
    "metric": {
        "name": ASLModel.VALID_ACCURACY,
        "goal": "maximize"
    },
    "parameters": {
        "lr": {
            "min": 1e-5,
            "max": 1e-3
        },
        "kernel_size": {
            "values": [3, 5, 7]
        },
        "dropout": {
            "min": 0.2,
            "max": 0.7
        },
        "hidden_dim": {
            "values": [256, 512, 1024],
        },
    }
}


In [7]:
def training_procedure():
    run = wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME)
    config = wandb.config
    run.name = f"ks{run.config.kernel_size}-lr{run.config.lr:.0e}"

    torch_model = ASLCNN_img(kernel_size=config.kernel_size, dropout=config.dropout, hidden_dim=config.hidden_dim)

    model = ASLModel(
        model=torch_model,
        criterion=nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam(torch_model.parameters(), lr=config.lr),
    )

    logger = WandbLogger(project=PROJECT_NAME, name=f"sweep_cnn_img_{wandb.run.name}")
    train(model, datamodule=datamodule, logger=logger)


In [8]:
sweep(sweep_config=sweep_config, count=25, training_procedure=training_procedure)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: u37g0me9
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/u37g0me9


wandb: Agent Starting Run: d059x1bc with config:
wandb: 	dropout: 0.20586427208804592
wandb: 	hidden_dim: 1024
wandb: 	kernel_size: 3
wandb: 	lr: 0.0001966223066672122
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shse13 (dspro2-silent-speech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:477: The total number of parameters detected may be inaccurate because the model contains an instance of `Un

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

wandb: Ctrl + C detected. Stopping sweep.


wandb: 
wandb: 🚀 View run desert-sweep-6 at: https://wandb.ai/dspro2-silent-speech/silent-speech/runs/124ku4lo
wandb: Find logs at: wandb/run-20250423_163646-124ku4lo/logs
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f40c33e8050>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f408ed41df0, execution_count=8 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f408ed42e70, raw_cell="sweep(sweep_config=sweep_config, count=25, trainin.." store_history=True silent=False shell_futures=True cell_id=536f42fa-8174-4ce1-84f1-31a7d6b35040> result=None>,),kwargs {}:


Exception in thread Thread-78 (_run_job):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5681/264292112.py", line 15, in training_procedure
  File "/home/jovyan/dspro2/models/training.py", line 39, in train
    trainer.fit(model, datamodule=datamodule)
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
         

BrokenPipeError: [Errno 32] Broken pipe

  ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 1012, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/trainer/trainer.py", line 1056, in _run_stage
    self.fit_loop.run()
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py", line 216, in run
    self.advance()
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py", line 455, in advance
self.epoch_loop.run(self._data_fetcher)
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/training_epoch_loop.py", line 150, in run
    self.advance(data_fetcher)
  File "/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/training_epoch_loop.py", l